# DATA PREPROCESSING AND DATA WRANGLING

Here I perform pre-processing Operations that are essential housekeeping before we fit our Machine Learning Models.

I also answer Question 3 which deals with Duplicate Transactions.

In [1]:
#Import Required Libraries
import numpy as np
import pandas as pd

#import json

In [2]:
#Load Data 
dataset = pd.read_pickle('data/pickles/df_new')
dataset.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,matchCVV
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,06/2023,2015-03-14,2015-03-14,1803,PURCHASE,0.0,False,False,False,1
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,02/2024,2015-03-14,2015-03-14,767,PURCHASE,0.0,True,False,False,1
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,08/2025,2015-03-14,2015-03-14,767,PURCHASE,0.0,False,False,False,1
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,08/2025,2015-03-14,2015-03-14,767,PURCHASE,0.0,False,False,False,1
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,10/2029,2015-08-06,2015-08-06,3143,PURCHASE,0.0,True,False,False,1


## Account Number , Customer Id and Card Last 4 digits

In [3]:

#The Two columns are similar and moreover as they are simply identifications, they will not influence Predictions
#We can drop them 

df_preprocess = dataset.copy()
df_preprocess.drop(['accountNumber','customerId'], axis=1, inplace=True)

#cardLast4Digits is also just a identification mark which is not useful in predictive modelling 
#We can drop the column
df_preprocess.drop(['cardLast4Digits'], axis=1, inplace=True)

## Acquired Country and Merchant Country Code 

In [4]:
#Check the two fields 
count_unique_1 = df_preprocess['acqCountry']. unique().size
count_unique_2 = df_preprocess['merchantCountryCode']. unique().size

print(count_unique_1,count_unique_2)



4 4


In [5]:
#The Two Fields both have similar values with only 4 uniques, I presume they will not affect our model
#Hence I will remove these fields entirely 

df_preprocess.drop(['acqCountry','merchantCountryCode'], axis=1, inplace=True)


## Dealing with Date Time 

- The Transaction Date Time Column is very important to this Dataset. I will convert the Datatime into an epoch.

- This will allow us to split the Date Time into chunks 

- There are also some relative Date Time which map the days since an event. I would like to represent them in Epoch Time as well.

In [6]:
#Here, I reformat the Transaction Date Time, I format the date into the form of epoch time 
#We also create new fields that will store each of our formatted dates : Year, Month, Day etc 

def DateTimeChange(df):    
    df["transDT"] = pd.to_datetime(df["transactionDateTime"])
    df["transMonth"] = df["transDT"].dt.month.astype(int)
    df["transDay"] = df["transDT"].dt.day.astype(int)
    df["transYear"] = df["transDT"].dt.year.astype(int)
    df["transHour"] = df["transDT"].dt.hour.astype(int)
    df["transactionformatted"] = df["transDT"].apply(lambda x: int(x.timestamp()))
    
    df.drop(['transactionDateTime','transDT'], axis=1, inplace=True)
    
    return df


In [7]:
#Here I repeat the above procedure for the relative dates in the dataframe that captures 'days since' type of dates

def rel_time_change(df):
    df["dateOfLastAddressChange"] = pd.to_datetime(df["dateOfLastAddressChange"])
    df["accountOpenDate"] = pd.to_datetime(df["accountOpenDate"])
    df["currentExpDate"] = pd.to_datetime(df["currentExpDate"])
    
    df["dateOfLastAddressChangeformatted"] = df["dateOfLastAddressChange"].apply(lambda x: int(x.timestamp()))
    df["accountOpenDateformatted"] = df["accountOpenDate"].apply(lambda x: int(x.timestamp()))
    df["currentExpDateformatted"] = df["currentExpDate"].apply(lambda x: int(x.timestamp()))
    
    df["timeTillExp"] = df["currentExpDateformatted"] - df["transactionformatted"]
    df["timeSinceAccountOpening"] = df["transactionformatted"] - df["accountOpenDateformatted"]
    df["timeSinceAddressChange"] = df["transactionformatted"] - df["dateOfLastAddressChangeformatted"]
    
    df.drop(['dateOfLastAddressChange','accountOpenDate','currentExpDate','dateOfLastAddressChangeformatted','accountOpenDateformatted','currentExpDateformatted'], axis=1, inplace=True)
    
    return df


In [8]:
#Apply the two functions to our data 

df_preprocess = DateTimeChange(df_preprocess)
df_preprocess = rel_time_change(df_preprocess)

## Dealing with Booleans

- Convert all boolean features to binary 1s and 0s


- cardPresent, expirationDateKeyInMatch, isFraud are the Boolean features we will convert to Binary

In [9]:
#Write a function that will take in the dataframe and convert the required Boolean features to 0s and 1s

def bool_to_binary(df):
    features = ['cardPresent', 'expirationDateKeyInMatch', 'isFraud']
    df[features] = df[features].astype(int)
    return df

df_preprocess = bool_to_binary(df_preprocess)

## One Hot-Encode Categorical Variables 

In [10]:
#Here I write a function to perform one hot encoding and then bind those features to our imput dataframe

def encode_and_bind(df, feature_to_encode):
    for feature in feature_to_encode:
        dummies = pd.get_dummies(df[feature])
        dummies.rename(columns=lambda x: feature + '_' + str(x), inplace=True)
        df = pd.concat([df, dummies], axis=1)        
        
    return(df) 

In [11]:
#Process the Merchant Name Column, Keep the name by deleting everything after the #, also remove trailing whitespace
df_preprocess["merchantName"] = df_preprocess["merchantName"].str.split('#').str[0].str.strip()

# define the list of categorical features to encode
feature_to_encode = ['posEntryMode', 'posConditionCode', 'merchantCategoryCode', 'transactionType', 'merchantName']

# one-hot encode the categorical features and bind the encoded features to the original DataFrame
df_preprocess = encode_and_bind(df_preprocess, feature_to_encode)

#Drop the original features after Encoding them 
df_preprocess.drop(feature_to_encode, axis=1, inplace = True)

In [12]:
df_preprocess

,creditLimit,availableMoney,transactionAmount,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,matchCVV,transMonth,transDay,...,merchantName_ikea.com,merchantName_netflix.com,merchantName_oldnavy.com,merchantName_pottery-barn.com,merchantName_sears.com,merchantName_staples.com,merchantName_target.com,merchantName_walmart.com,merchantName_westelm.com,merchantName_williamssonoma.com
0,5000,5000.00,98.55,0.00,0,0,0,1,8,13,...,0,0,0,0,0,0,0,0,0,0
1,5000,5000.00,74.51,0.00,1,0,0,1,10,11,...,0,0,0,0,0,0,0,0,0,0
2,5000,5000.00,7.47,0.00,0,0,0,1,11,8,...,0,0,0,0,0,0,0,0,0,0
3,5000,5000.00,7.47,0.00,0,0,0,1,12,10,...,0,0,0,0,0,0,0,0,0,0
4,5000,5000.00,71.18,0.00,1,0,0,1,3,24,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,50000,48904.96,119.92,1095.04,0,0,0,1,12,22,...,0,0,0,0,0,0,0,0,0,0
786359,50000,48785.04,18.89,1214.96,0,0,0,1,12,25,...,0,0,0,0,0,0,0,0,0,0
786360,50000,48766.15,49.43,1233.85,0,0,0,1,12,27,...,0,0,0,0,0,0,0,0,0,0
786361,50000,48716.72,49.89,1283.28,0,0,0,1,12,29,...,0,0,0,0,0,0,0,1,0,0


In [13]:
print("Dataset Dimensions: {} rows x {} columns".format(df_preprocess.shape[0], df_preprocess.shape[1]))

Dataset Dimensions: 786363 rows x 251 columns


In [14]:
#save preprocessed data
pd.to_pickle(df_preprocess, 'data/pickles/df_preprocess', protocol=4)

### As Expected following the Encoding of Variables our Dataset has grown considerably in Dimensions.



# QUESTION 3 - WRANGLING

#### I am identifying the reversed transactions by the transaction type that is REVESAL.

#### If the transaction type is not REVERSAL and the type elapsed between two consecutive transactions is 5 minutes (This is an arbitrary assumption I make, could vary) then I classify the transaction as Multi-Swipe.



In [15]:
df_duplicates = dataset.copy()

In [16]:
#Apply the prePreocess function we wrote earlier to convert our DateTime column into an epoch format
df_duplicates = DateTimeChange(df_duplicates)

In [17]:
#Sort the DataFrame by Account Number and Reset it's index

df_duplicates = df_duplicates.sort_values(['accountNumber'])
df_duplicates = df_duplicates.reset_index(drop=True)

In [18]:
#Set a time limit which decides if a transaction is a Multiswipe. 
#I will assume a time limit of 5 Minutes

time_limit = 300 #300 seconds i.e 5 minutes

In [19]:
#Group values in our Data Frame by Account Number 
df_grouped = df_duplicates.groupby('accountNumber')

In [20]:
#Empty List to store the reversed and multiswipe transactions

reversed_transactions = []
multiswipe_transactions = []

#### The Below is a computationally very expensive process as it involves running nested loops on large dataframes. 

- I tried to do the same with vectorized Data Frames but the results were not satisfactory




In [21]:
for accNo, df in df_grouped:
    df = df.sort_values(['transactionAmount'])
    df = df.reset_index(drop=True)
    
    # Loop through each transaction, comparing it to every other transaction with a higher index
    for i in range(0,len(df["transactionAmount"])-1):
        for j in range(i+1,len(df["transactionAmount"])-1):
            
            # If two transactions have the same amount, calculate the time difference between them
            if df_duplicates ["transactionAmount"][i] == df_duplicates ["transactionAmount"][j]:
                
                time_difference = abs(df_duplicates ["transactionformatted"][j] - df_duplicates ["transactionformatted"][i])
            
                # If one transaction is a REVERSAL and the other is not,
                #consider the REVERSAL transaction as a potential Duplicate
                if (df_duplicates ["transactionType"][j] == "REVERSAL" and \
                    df_duplicates ["transactionType"][i] != "REVERSAL") \
                or (df_duplicates ["transactionType"][j] != "REVERSAL" \
                    and df_duplicates ["transactionType"][i] == "REVERSAL"):
                    
                    reversed_transactions.append(df_duplicates ["transactionAmount"][j])

            
                # If the time difference between two transactions is less than or equal to a certain limit, it is a duplicate               
                elif time_difference <= time_limit:
                    
                    multiswipe_transactions.append(df_duplicates ["transactionAmount"][j])
                    
            # The Transactions are sorted by Amount, if the amount is lesser, break out of the loop            
            elif df_duplicates ["transactionAmount"][i] < df_duplicates ["transactionAmount"][j]:
                
                break

In [22]:
reversed_count = len(reversed_transactions)
reversed_total = round(np.array(reversed_transactions).sum(),4)
multiswipe_count = len(multiswipe_transactions)
multiswipe_total = round(np.array(multiswipe_transactions).sum(),4)



print("There are total of {} reversed transactions amounting to ${}".format(reversed_count, reversed_total))
print("There are total of {} multiswipe transactions amounting to ${}".format(multiswipe_count, multiswipe_total))

There are total of 3501 reversed transactions amounting to $640960.33
There are total of 7649 multiswipe transactions amounting to $2690437.35
